<a href="https://colab.research.google.com/github/harenlin/PySpark-Learning/blob/main/NLP_PySpark_Basic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# ref: NLP_in_Pysparks_MLlib.ipynb
!pip install pyspark
import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("NLP").getOrCreate()
cores = spark._jsc.sc().getExecutorMemoryStatus().keySet().size()
print("You are working with", cores, "core(s)")
spark

     |████████████████████████████████| 212.4MB 68kB/s 
     |████████████████████████████████| 204kB 16.3MB/s 
  Created wheel for pyspark: filename=pyspark-3.1.2-py2.py3-none-any.whl size=212880768 sha256=9d793d892e7cf9e71920390f498bcf2819ac4204af9e2e544ae5c16512deed62
  Stored in directory: /root/.cache/pip/wheels/40/1b/2c/30f43be2627857ab80062bef1527c0128f7b4070b6b2d02139
Successfully built pyspark
You are working with 1 core(s)


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# NLP in Pyspark's MLlib

Natural Language Processing (NLP) is a very trendy topic in the data science area today that is really handy for tasks like **chat bots**, movie or **product review analysis** and especially **tweet classification**. In this notebook, we will cover the **classification aspect of NLP** and go over the features that Spark has for cleaning and preparing your data for analysis. We will also touch on how to implement **ML Pipelines** to a few of our data processing steps to help make our code run a bit faster. 

As we learned in the NLP concept review lectures, the text you process must first be cleaned, tokenized and vectorized. Essentially, we need to covert our text into a vector of numbers. But how do we do that? Spark has a variety of built in functions to accomplish all of these tasks very easily. We will cover all of it here!

### Agenda

    1. Review Data (quality check)
    2. Clean up the data (remove puncuation, special characters, etc.)
    3. Tokenize text data
    4. Remove Stopwords
    5. Zero index our label column
    5. Create an ML Pipeline (to streamline steps 3-5)
    6. Vectorize Text column
         - Count Vectors
         - TF-IDF
         - Word2Vec
    7. Train and Evaluate Model (classification)
    8. View Predictions

In [ ]:
from pyspark.ml.feature import * # CountVectorizer, StringIndexer, RegexTokenizer, StopWordsRemover
from pyspark.sql.functions import * # col, udf, regexp_replace, isnull
from pyspark.sql.types import * # StringType, IntegerType
from pyspark.ml.classification import *
from pyspark.ml.evaluation import *
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder

# For pipeline development
from pyspark.ml import Pipeline 

In [ ]:
path = '/content/drive/My Drive/PySpark/Datasets/'
df = spark.read.csv(path + 'kickstarter.csv', inferSchema=True, header=True)

In [ ]:
df.printSchema()

root
 |-- _c0: string (nullable = true)
 |-- blurb: string (nullable = true)
 |-- state: string (nullable = true)



In [ ]:
df.show(5,False)

+---+-----------------------------------------------------------------------------------------------------------------------------------+----------+
|_c0|blurb                                                                                                                              |state     |
+---+-----------------------------------------------------------------------------------------------------------------------------------+----------+
|1  |Using their own character, users go on educational quests around a virtual world leveling up subject-oriented skills (ie Physics). |failed    |
|2  |MicroFly is a quadcopter packed with WiFi, 6 sensors, and 3 processors for ultimate stability -- and fits in the palm of your hand.|successful|
|3  |A small indie press, run as a collective for authors who want to self-publish, and a sexy, smart , hilarious novel!                |failed    |
|4  |Zylor is a new baby cosplayer! Back this kickstarter to help fund new cosplay photoshoots to share hi

In [ ]:
# how many rows
df.count()

223627

In [ ]:
# How many null values do we have?
from pyspark.sql.functions import *

def null_value_calc(df):
    null_columns_counts = []
    numRows = df.count()
    for k in df.columns:
        nullRows = df.where(col(k).isNull()).count()
        if(nullRows > 0):
            temp = k, nullRows, (nullRows/numRows)*100
            null_columns_counts.append(temp)
    return null_columns_counts

null_columns_calc_list = null_value_calc(df)
spark.createDataFrame(null_columns_calc_list, ['Column_Name', 'Null_Values_Count','Null_Value_Percent']).show()

+-----------+-----------------+------------------+
|Column_Name|Null_Values_Count|Null_Value_Percent|
+-----------+-----------------+------------------+
|      blurb|             1488|0.6653937136392296|
|      state|            13157| 5.883457722010312|
+-----------+-----------------+------------------+



In [ ]:
print(df.na.drop().count())
df = df.dropna()
df.count()

210470


210470

In [ ]:
# Slice the dataframe down to make this notebook run faster
df = df.limit(500)
print('Sliced row count:', df.count())

Sliced row count: 500


## Quality Assurance Check (QA)

Let's make sure our dependent variable column is clean before we go any further. This an important step in our analysis. 

In [ ]:
# Quick data quality check on the state column....
# This is going to be our category column so it's important
df.groupBy("state").count().orderBy(col("count").desc()).show(10,False)

+---------------------------------------------------------------------------+-----+
|state                                                                      |count|
+---------------------------------------------------------------------------+-----+
|failed                                                                     |337  |
|successful                                                                 |156  |
| best experienced with Oculus Rift."                                       |1    |
| mixing pixel art graphics with deep storytelling and action combat system"|1    |
| smart                                                                     |1    |
| with the editor                                                           |1    |
| Adventure with RPG elements."                                             |1    |
| from Tinder                                                               |1    |
|"" with a special focus on Ty Cobb."                                       

In [ ]:
df = df.filter("state IN('successful','failed')")
# Make sure it worked
df.groupBy("state").count().orderBy(col("count").desc()).show(truncate=False)

+----------+-----+
|state     |count|
+----------+-----+
|failed    |337  |
|successful|156  |
+----------+-----+



In [ ]:
df.show()

+---+--------------------+----------+
|_c0|               blurb|     state|
+---+--------------------+----------+
|  1|Using their own c...|    failed|
|  2|MicroFly is a qua...|successful|
|  3|A small indie pre...|    failed|
|  4|Zylor is a new ba...|    failed|
|  5|Hatoful Boyfriend...|    failed|
|  6|FastMan is a Infi...|    failed|
|  7|FADE. A dark and ...|    failed|
|  8|The next generati...|    failed|
|  9|Whip around plane...|    failed|
| 10|Sneak in, find tr...|    failed|
| 11|A unique 3rd pers...|    failed|
| 12|Echo-S is a mod t...|    failed|
| 13|Game development ...|    failed|
| 14|GangWars ARC is a...|    failed|
| 15|A team based firs...|    failed|
| 16|Super Babies: Wor...|    failed|
| 17|Action RPG set in...|    failed|
| 19|Studly the Muffin...|    failed|
| 20|A new YouTube web...|    failed|
| 21|Experience rock c...|    failed|
+---+--------------------+----------+
only showing top 20 rows



In [ ]:
# Let's check the quality of the blurbs
df.select("blurb").show(10,False)

+-----------------------------------------------------------------------------------------------------------------------------------+
|blurb                                                                                                                              |
+-----------------------------------------------------------------------------------------------------------------------------------+
|Using their own character, users go on educational quests around a virtual world leveling up subject-oriented skills (ie Physics). |
|MicroFly is a quadcopter packed with WiFi, 6 sensors, and 3 processors for ultimate stability -- and fits in the palm of your hand.|
|A small indie press, run as a collective for authors who want to self-publish, and a sexy, smart , hilarious novel!                |
|Zylor is a new baby cosplayer! Back this kickstarter to help fund new cosplay photoshoots to share his cuteness with the world!    |
|Hatoful Boyfriend meet Skeletons! A comedy Dating Sim that pu

In [ ]:
# Replace Slashes and parenthesis with spaces
# You can test your script on line 7 of the df "(Legend of Zelda/Fable Inspired)"
df = df.withColumn("blurb", translate(col("blurb"), "/", " ")) \
       .withColumn("blurb", translate(col("blurb"), "(", " ")) \
       .withColumn("blurb", translate(col("blurb"), ")", " "))
df.select("blurb").show(10, False)

+-----------------------------------------------------------------------------------------------------------------------------------+
|blurb                                                                                                                              |
+-----------------------------------------------------------------------------------------------------------------------------------+
|Using their own character, users go on educational quests around a virtual world leveling up subject-oriented skills  ie Physics . |
|MicroFly is a quadcopter packed with WiFi, 6 sensors, and 3 processors for ultimate stability -- and fits in the palm of your hand.|
|A small indie press, run as a collective for authors who want to self-publish, and a sexy, smart , hilarious novel!                |
|Zylor is a new baby cosplayer! Back this kickstarter to help fund new cosplay photoshoots to share his cuteness with the world!    |
|Hatoful Boyfriend meet Skeletons! A comedy Dating Sim that pu

In [ ]:
# Removing anything that is not a letter
df = df.withColumn("blurb", regexp_replace(col('blurb'), '[^A-Za-z ]+', ''))
df.select("blurb").show(10, False)

+-------------------------------------------------------------------------------------------------------------------------------+
|blurb                                                                                                                          |
+-------------------------------------------------------------------------------------------------------------------------------+
|Using their own character users go on educational quests around a virtual world leveling up subjectoriented skills  ie Physics |
|MicroFly is a quadcopter packed with WiFi  sensors and  processors for ultimate stability  and fits in the palm of your hand   |
|A small indie press run as a collective for authors who want to selfpublish and a sexy smart  hilarious novel                  |
|Zylor is a new baby cosplayer Back this kickstarter to help fund new cosplay photoshoots to share his cuteness with the world  |
|Hatoful Boyfriend meet Skeletons A comedy Dating Sim that puts you into a high school ful

In [ ]:
# Remove multiple spaces
df = df.withColumn("blurb", regexp_replace(col('blurb'), ' +', ' '))
df.select("blurb").show(10, False)

+------------------------------------------------------------------------------------------------------------------------------+
|blurb                                                                                                                         |
+------------------------------------------------------------------------------------------------------------------------------+
|Using their own character users go on educational quests around a virtual world leveling up subjectoriented skills ie Physics |
|MicroFly is a quadcopter packed with WiFi sensors and processors for ultimate stability and fits in the palm of your hand     |
|A small indie press run as a collective for authors who want to selfpublish and a sexy smart hilarious novel                  |
|Zylor is a new baby cosplayer Back this kickstarter to help fund new cosplay photoshoots to share his cuteness with the world |
|Hatoful Boyfriend meet Skeletons A comedy Dating Sim that puts you into a high school full of Sk

In [ ]:
# Lower case everything
df = df.withColumn("blurb", lower(col('blurb')))
df.select("blurb").show(10, False)

+------------------------------------------------------------------------------------------------------------------------------+
|blurb                                                                                                                         |
+------------------------------------------------------------------------------------------------------------------------------+
|using their own character users go on educational quests around a virtual world leveling up subjectoriented skills ie physics |
|microfly is a quadcopter packed with wifi sensors and processors for ultimate stability and fits in the palm of your hand     |
|a small indie press run as a collective for authors who want to selfpublish and a sexy smart hilarious novel                  |
|zylor is a new baby cosplayer back this kickstarter to help fund new cosplay photoshoots to share his cuteness with the world |
|hatoful boyfriend meet skeletons a comedy dating sim that puts you into a high school full of sk

In [ ]:
# Tokenize
regex_tokenizer = RegexTokenizer(inputCol="blurb", outputCol="words", pattern="\\W") # These also work as well: "\W", r"\W"
raw_words = regex_tokenizer.transform(df)
print(raw_words.show(5,False))

# Remove Stop words
remover = StopWordsRemover(inputCol="words", outputCol="filtered")
words_df = remover.transform(raw_words)
print(words_df.show(5,False))

# Zero Index Label Column
indexer = StringIndexer(inputCol="state", outputCol="label")
feature_data = indexer.fit(words_df).transform(words_df)
print(feature_data.show(5,False))

+---+------------------------------------------------------------------------------------------------------------------------------+----------+----------------------------------------------------------------------------------------------------------------------------------------------------+
|_c0|blurb                                                                                                                         |state     |words                                                                                                                                               |
+---+------------------------------------------------------------------------------------------------------------------------------+----------+----------------------------------------------------------------------------------------------------------------------------------------------------+
|1  |using their own character users go on educational quests around a virtual world leveling up subjectoriented skills i

In [ ]:
################# PIPELINED ##################
# Tokenize
regex_tokenizer = RegexTokenizer(inputCol="blurb", outputCol="words", pattern="\\W")
# raw_words = regex_tokenizer.transform(df)

# Remove Stop words
remover = StopWordsRemover(inputCol=regex_tokenizer.getOutputCol(), outputCol="filtered")
# words_df = remover.transform(raw_words)

# Zero Index Label Column
indexer = StringIndexer(inputCol="state", outputCol="label")
# feature_data = indexer.fit(words_df).transform(words_df)

# Create the Pipeline
pipeline = Pipeline(stages=[regex_tokenizer,remover,indexer])
data_prep_pl = pipeline.fit(df)
# print(type(data_prep_pl))
# print(" ")
# Now call on the Pipeline to get our final df
feature_data = data_prep_pl.transform(df)
feature_data.show(5,False)

+---+------------------------------------------------------------------------------------------------------------------------------+----------+----------------------------------------------------------------------------------------------------------------------------------------------------+--------------------------------------------------------------------------------------------------------------------------+-----+
|_c0|blurb                                                                                                                         |state     |words                                                                                                                                               |filtered                                                                                                                  |label|
+---+------------------------------------------------------------------------------------------------------------------------------+----------+-------------

## Converting text into vectors

We will test out the following three vectorizors:

1. Count Vectors
2. TF-IDF
3. Word2Vec

In [ ]:
# Count Vector (count vectorizer and hashingTF are basically the same thing)
# cv = CountVectorizer(inputCol="filtered", outputCol="features")
# model = cv.fit(feature_data)
# countVectorizer_features = model.transform(feature_data)

In [ ]:
# Hashing TF (Transformer)
hashingTF = HashingTF(inputCol="filtered", outputCol="rawfeatures", numFeatures=200)
HTFfeaturizedData = hashingTF.transform(feature_data)
print(HTFfeaturizedData.show(5, False))

+---+------------------------------------------------------------------------------------------------------------------------------+----------+----------------------------------------------------------------------------------------------------------------------------------------------------+--------------------------------------------------------------------------------------------------------------------------+-----+-----------------------------------------------------------------------------------------------------+
|_c0|blurb                                                                                                                         |state     |words                                                                                                                                               |filtered                                                                                                                  |label|rawfeatures                                            

In [ ]:
# rename the HTF features to features to be consistent
HTFfeaturizedData = HTFfeaturizedData.withColumnRenamed("rawfeatures", "features")
HTFfeaturizedData.name = 'HTFfeaturizedData' #We will use later for printing

In [ ]:
# TF-IDF (IDF: Estimator)
idf = IDF(inputCol="rawfeatures", outputCol="features")
idfModel = idf.fit(HTFfeaturizedData)
TFIDFfeaturizedData = idfModel.transform(HTFfeaturizedData)
print(TFIDFfeaturizedData.show(5, False))
TFIDFfeaturizedData.name = 'TFIDFfeaturizedData'
TFIDFfeaturizedData.name

+---+------------------------------------------------------------------------------------------------------------------------------+----------+----------------------------------------------------------------------------------------------------------------------------------------------------+--------------------------------------------------------------------------------------------------------------------------+-----+-----------------------------------------------------------------------------------------------------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|_c0|blurb                                                                                                                         |state     |words                                                  

'TFIDFfeaturizedData'

In [ ]:
# Word2Vec
word2Vec = Word2Vec(vectorSize=100, minCount=0, inputCol="filtered", outputCol="features")
model = word2Vec.fit(feature_data)
W2VfeaturizedData = model.transform(feature_data)
W2VfeaturizedData.show(5,False)

+---+------------------------------------------------------------------------------------------------------------------------------+----------+----------------------------------------------------------------------------------------------------------------------------------------------------+--------------------------------------------------------------------------------------------------------------------------+-----+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [ ]:
# W2Vec Dataframes typically has negative values so we will correct for that here so that we can use the Naive Bayes classifier
scaler = MinMaxScaler(inputCol="features", outputCol="scaledFeatures")
# Compute summary statistics and generate MinMaxScalerModel
scalerModel = scaler.fit(W2VfeaturizedData)
# rescale each feature to range [min, max].
scaled_data = scalerModel.transform(W2VfeaturizedData)
scaled_data.show(5,False)

+---+------------------------------------------------------------------------------------------------------------------------------+----------+----------------------------------------------------------------------------------------------------------------------------------------------------+--------------------------------------------------------------------------------------------------------------------------+-----+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [ ]:
W2VfeaturizedData = scaled_data.select('state', 'blurb', 'label', 'scaledFeatures')
W2VfeaturizedData = W2VfeaturizedData.withColumnRenamed('scaledFeatures','features')
W2VfeaturizedData.name = 'W2VfeaturizedData' # We will need this to print later
W2VfeaturizedData.show(5,False)

+----------+------------------------------------------------------------------------------------------------------------------------------+-----+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

# Modeling

In [ ]:
final_data = W2VfeaturizedData.select('label', 'features', 'blurb')
final_data.show(5,False)

+-----+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [ ]:
train, test = final_data.randomSplit([0.7,0.3])

In [ ]:
# Set up our evaluation objects
Bin_evaluator = BinaryClassificationEvaluator(rawPredictionCol='prediction') #labelCol='label'
# Bin_evaluator = BinaryClassificationEvaluator() #labelCol='label'
MC_evaluator = MulticlassClassificationEvaluator(metricName="accuracy") # redictionCol="prediction",

In [ ]:
# This is the most simplistic approach which does not use cross validation
# Let's go ahead and train a Logistic Regression Algorithm
classifier = LogisticRegression()
fitModel = classifier.fit(train)

# Evaluation method for binary classification problem
predictionAndLabels = fitModel.transform(test)
auc = Bin_evaluator.evaluate(predictionAndLabels) # auc = area under roc curve
print("AUC: ", auc) 

# Evaluation for a multiclass classification problem
predictions = fitModel.transform(test)
accuracy = (MC_evaluator.evaluate(predictions))*100
print("Accuracy: {0:.2f}".format(accuracy),"%") #     print("Test Error = %g " % (1.0 - accuracy))
print(" ")

AUC:  0.6278766825879288
Accuracy: 67.13 %
 
